# CARLA Project - Car and distance detection

### Package installation

Install the required packages with the following command (run it where the `requirements.txt` file is located):

```bash
pip install -r requirements.txt
```

In [ ]:
import carla, cv2
import numpy as np
from queue import Queue
from queue import Empty
from ultralytics import YOLO
import paho.mqtt.client as mqtt

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [ ]:
def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.5, z=1.5), carla.Rotation(pitch=-10)), width=800, height=600, frequency = 0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('sensor_tick', str(frequency))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=1.5, z=1.5), carla.Rotation(pitch=-10)), horizontal_fov = 35, vertical_fov = 20):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar

In [ ]:
# Clean the world from previous actors
for actor in world.get_actors().filter('*vehicle*'):
  actor.destroy()

for actor in world.get_actors().filter('*sensor*'):
  actor.destroy()

In [ ]:
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# -------------- Connection to MQTT broker --------------
broker_address = "localhost"  # Change to your broker address
port = 1883  # Default MQTT port
topic = "car_distance"

client = mqtt.Client()
client.connect(broker_address, port)

# Load the YOLO model
model = YOLO("runs/detect/train2/weights/best.pt")

# Sensors configuration
img_size = 320
transforms = [
    carla.Transform(carla.Location(x=-0.16, y=-0.9, z=2.4), carla.Rotation(yaw=-100)),  # [0] Left side camera  
    carla.Transform(carla.Location(x=-0.16, y=0.9, z=2.4), carla.Rotation(yaw=100)),    # [1] Right side camera
    carla.Transform(carla.Location(x=1.5, z=2.4)),                                      # [2] Front camera
    carla.Transform(carla.Location(x=-1.5, z=2.4), carla.Rotation(yaw=180))             # [3] Rear camera
]

vehicle = spawn_vehicle(spawn_index=10, pattern="vehicle.dodge.charger_2020")

video_outputs = [np.zeros((img_size, img_size, 4), dtype=np.uint8) for _ in range(4)]

# Thread-safe queues to store the sensor data
image_queues = [Queue() for _ in range(4)]
radar_queues = [Queue() for _ in range(4)]

def sensor_callback(data, queue):
    queue.put(data)

cameras = []
radars = []

for i in range(4):
    camera = spawn_camera(attach_to=vehicle, transform=transforms[i], width=img_size, height=img_size)
    radar = spawn_radar(attach_to=vehicle, transform=transforms[i], horizontal_fov=60, vertical_fov=40)
    camera.listen(lambda data, idx=i: sensor_callback(data, image_queues[idx]))
    radar.listen(lambda data, idx=i: sensor_callback(data, radar_queues[idx]))
    cameras.append(camera)
    radars.append(radar)

vehicle.set_autopilot(True)

#Windows configuration
window_names = ['Left Side Camera', 'Right Side Camera', 'Front Camera', 'Rear Camera']

for i in range(4):
    cv2.namedWindow(window_names[i], cv2.WINDOW_AUTOSIZE)

# Build the K projection matrix: - used later to project the radar points into the image plane
# K = [[Fx,  0, image_w/2],
#      [ 0, Fy, image_h/2],
#      [ 0,  0,         1]]
# image_w = image_h = img_size 

#since we are using the standard camera fov
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
fov = camera_bp.get_attribute("fov").as_float() 
focal = img_size / (2.0 * np.tan(fov * np.pi / 360.0))

# In this case Fx and Fy are the same since the pixel aspect ratio is 1
K = np.identity(3)
K[0, 0] = K[1, 1] = focal
K[0, 2] = img_size / 2.0
K[1, 2] = img_size / 2.0

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        
        world.tick()

        for k in range(4): # k = for each camera
            try:
                # Get the data once it's received.
                image_data = image_queues[k].get(True, 1.0)
                radar_data = radar_queues[k].get(True, 1.0)
            except Empty:
                print("[Warning] Some sensor data has been missed")
                continue

            # sync the data
            assert image_data.frame == radar_data.frame

            # Retrive the image data
            video_outputs[k] = np.reshape(np.copy(image_data.raw_data), (image_data.height, image_data.width, 4))

            # ----------- RADAR data -----------

            current_rot = radar_data.transform.rotation

            points = np.frombuffer(radar_data.raw_data, dtype=np.dtype('f4'))
            points = np.reshape(points, (len(radar_data), 4))

            # [vel, azimuth, altitude, depth]
            azimuth = points[:, 1]
            altitude = points[:, 2]
            depth = points[:, 3]

            # Convert spherical coordinates to Cartesian (sensor coordinate system)
            X = depth * np.cos(altitude) * np.cos(azimuth)
            Y = depth * np.cos(altitude) * np.sin(azimuth)
            Z = depth * np.sin(altitude)

            # Stack into a single array (shape: Nx3)
            cartesian_points = np.stack((X, Y, Z), axis=1)

            cartesian_points = cartesian_points.T # Transpose to 3xN

            # change from UE4's coordinate system to an "standard" camera coordinate system (the same used by OpenCV):
            point_in_std_camera_coords = np.array([
                    cartesian_points[1],
                    cartesian_points[2] * -1,
                    cartesian_points[0]])

            # Project the 3D points into the 2D image with the K matrix
            points_2d = np.dot(K, point_in_std_camera_coords)

            # Normalize the points
            points_2d = np.array([
                points_2d[0, :] / points_2d[2, :],
                points_2d[1, :] / points_2d[2, :],
                points_2d[2, :]])
            
            points_2d = points_2d.T

            frame = video_outputs[k].copy()  # Copy to avoid modifying the original

            # ----------- YOLO evaluation -----------

            eval_img = frame[:, :, :3]
            results = model(eval_img, imgsz=img_size)

            for result in results:
                for i in range(len(result)): # i = for each detected object
                    min_distance = -1
                    confidence = result.boxes.conf[i]
                    if confidence < 0.5: # skip if confidence is low
                        continue
                    box = result.boxes.xyxy[i]
                    cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 1)

                    for j in range(len(points_2d)): # j = for each radar point
                        if min_distance == -1:
                            min_distance = depth[j]

                        x, y = int(points_2d[j][0]), int(points_2d[j][1])
                        if box[0] < x < box[2] and box[1] < y < box[3]:
                            cv2.circle(frame, (x, y), radius=2, color=(0, 0, 255), thickness=-1)
                            if depth[j] < min_distance:
                                min_distance = depth[j]
                    if min_distance != -1:
                        message = f"{window_names[k]} - Car [{i}] detected at distance: {min_distance}"
                        client.publish(topic, message)
                        # print(window_names[k]," - Car [", i ,"] detected at distance: ", min_distance)
                    else:
                        print("[Warning] ",window_names[k]," - no radar data on car")
            
            cv2.imshow(window_names[k], frame)
finally:
    client.disconnect()
    cv2.destroyAllWindows()
    vehicle.destroy()
    for camera in cameras:
        camera.destroy()
    for radar in radars:
        radar.destroy()
    settings = world.get_settings()
    settings.synchronous_mode = False
    settings.no_rendering_mode = False
    settings.fixed_delta_seconds = None
    world.apply_settings(settings)